# Model Metadata API Demo

The Model Metadata API is still in developement. This demo showcases some of the functionality of the new API.

**Note**: If you are running with self-signed certificate make sure to set `CURL_CA_BUNDLE` environmental variable before executing this notebook, for example start the notebook server with:
```bash
CURL_CA_BUNDLE="" jupyter lab
```

## API Configuration

In [1]:
from seldon_deploy_sdk.rest import ApiException
from seldon_deploy_sdk import V1Model, ModelMetadataServiceApi, Configuration, ApiClient
from seldon_deploy_sdk.auth import OIDCAuthenticator

In [2]:
from pprint import pprint
import json

In [3]:
# For local testing use
INGRESS = "http://localhost:8080"

# For production usecase set this to your ingress
# for example if your SD URL is http://xxx.yyy.zzz.xyz/seldon-deploy/api/v1alpha1 set
# INGRESS = "http://xxx.yyy.zzz.xyz"

In [4]:
config = Configuration()
config.host = f"{INGRESS}/seldon-deploy/api/v1alpha1"
config.verify_ssl = False

In [5]:
# auth if needed
config.oidc_server = f"{INGRESS}/auth/realms/deploy-realm"
config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = "client_credentials"

# Alternativey set instead of the "client_credentials" flow
# config.username = "admin@seldon.io"
# config.password = "12341234"
# config.auth_method = "password"

auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()

In [6]:
api_client = ApiClient(configuration=config, authenticator=auth)
api_instance = ModelMetadataServiceApi(api_client)

## Add single model to the Model Catalogue

In [7]:
model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-test-model-alpha-repo",
        "an arbitrary tag": "true",
    },
)
try:
    # Create a Model Metadata entry.
    api_response = api_instance.model_metadata_service_create_model_metadata(model)
    pprint(api_response)
except ApiException as e:
    print(f"Couldn't create model: {json.loads(e.body)['message']}")

Couldn't create model: code=AlreadyExists desc=ERROR: duplicate key value violates unique constraint "model_uri" (SQLSTATE 23505)


## Add multiple models to the Model Catalogue

In [8]:
models = [
    #     Same model different versions
    {
        "uri": "gs://test-model-beta-v1.0.0",
        "name": "beta",
        "version": "v1.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Jon"},
    },
    {
        "uri": "gs://test-model-beta-v2.0.0",
        "name": "beta",
        "version": "v2.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
    {
        "uri": "gs://test-model-beta-v3.0.0",
        "name": "beta",
        "version": "v3.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
]

for model in models:
    body = V1Model(**model)
    try:
        api_response = api_instance.model_metadata_service_create_model_metadata(body)
        pprint(api_response)
    except ApiException as e:
        print(f"Couldn't create model: {json.loads(e.body)['message']}")

{}
{}
{}


## List all models in the Model Catalogue

In [9]:
try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata()
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'models': [{'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2024, 2, 7, 10, 26, 22, 30282, tzinfo=tzutc()),
             'metrics': {},
             'name': '',
             'prediction_schema': None,
             'project': 'default',
             'runtime_defaults': {'default_protocol': 'PROTOCOL_SELDON'},
             'tags': {'auto_created': 'true'},
             'task_type': '',
             'uri': 'gs://seldon-models/v1.18.0/sklearn/iris',
             'version': ''},
            {'artifact_type': 'TENSORFLOW',
             'creation_time': datetime.datetime(2024, 2, 7, 10, 28, 18, 256331, tzinfo=tzutc()),
             'metrics': {},
             'name': '',
             'prediction_schema': None,
             'project': 'default',
             'runtime_defaults': {'default_protocol': 'PROTOCOL_TENSORFLOW'},
             'tags': {'auto_created': 'true'},
             'task_type': '',
             'uri': 'gs://seldon-models/tfserving/cifar10/resnet32',

## Get all models with a specific name and authored by Jon (name = "beta" and tags.author = "Jon")

In [10]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# artifactType = 'artifactType_example'     # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(
        name="beta", tags={"author": "Jon"}
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'models': [{'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2024, 2, 7, 14, 45, 1, 169717, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'prediction_schema': None,
             'project': 'default',
             'runtime_defaults': None,
             'tags': {'author': 'Jon'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}


## Get all models authored by Bob (tags.author = Bob)

In [11]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# artifactType = 'artifactType_example' # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(
        tags={"author": "Bob"}
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'models': [{'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2024, 2, 7, 14, 45, 1, 236044, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'prediction_schema': None,
             'project': 'default',
             'runtime_defaults': None,
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v2.0.0',
             'version': 'v2.0.0'},
            {'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2024, 2, 7, 14, 45, 1, 321901, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'prediction_schema': None,
             'project': 'default',
             'runtime_defaults': None,
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v3.0.0',
             'version': 'v3.0.0'}],
 'next_page_token': ''}


## Update metadata for a model in the Model Catalog by adding an additional tag

In [12]:
try:
    # Get Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(
        uri="gs://test-model-alpha-v1.0.0"
    )
    print("Before update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")


model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-other-repo",
        "an arbitrary tag": "true",
        "an additional tag": "123",
    },
)

try:
    # Update a Model Metadata entry.
    api_response = api_instance.model_metadata_service_update_model_metadata(model)
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(
        uri="gs://test-model-alpha-v1.0.0"
    )
    print("After update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Before update:
{'models': [{'artifact_type': 'XGBOOST',
             'creation_time': datetime.datetime(2024, 2, 7, 14, 44, 18, 265512, tzinfo=tzutc()),
             'metrics': {},
             'name': 'alpha',
             'prediction_schema': None,
             'project': 'default',
             'runtime_defaults': None,
             'tags': {'an arbitrary tag': 'true',
                      'source': 'https://github.com/some-test-model-alpha-repo'},
             'task_type': 'regression',
             'uri': 'gs://test-model-alpha-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}
{}
After update:
{'models': [{'artifact_type': 'XGBOOST',
             'creation_time': datetime.datetime(2024, 2, 7, 14, 44, 18, 265512, tzinfo=tzutc()),
             'metrics': {},
             'name': 'alpha',
             'prediction_schema': None,
             'project': 'default',
             'runtime_defaults': None,
             'tags': {'an additional tag': '123',
              

## Get runtime information for a model

You will need a Seldon Deployment running with the following details in order to receive a meaningful result:

In [13]:
name = "iris"
namespace = "seldon"
model_uri = "gs://seldon-models/v1.18.0/sklearn/iris"

### List runtime metadata for a specific model run a "(deployment_status = "Running")

In [14]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        model_uri=model_uri, deployment_status="Running"
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '',
 'runtime_metadata': [{'creation_time': datetime.datetime(2024, 2, 7, 10, 27, 16, 79854, tzinfo=tzutc()),
                       'deployment_kubernetes_uid': '0474d68d-db43-4b66-b3cc-14f6d7ca02ab',
                       'deployment_name': 'mc1-iris',
                       'deployment_namespace': 'seldon',
                       'deployment_status': 'Running',
                       'deployment_type': 'SeldonDeployment',
                       'explainer': False,
                       'model': {'artifact_type': 'SKLEARN',
                                 'creation_time': datetime.datetime(2024, 2, 7, 10, 26, 22, 30282, tzinfo=tzutc()),
                                 'metrics': {},
                                 'name': '',
                                 'prediction_schema': None,
                                 'project': 'default',
                                 'runtime_defaults': {'default_protocol': 'PROTOCOL_SELDON'},
                            

### List runtime metadata for models in a specific deployment and namespace

In [15]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        deployment_name=name, deployment_namespace=namespace
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '', 'runtime_metadata': []}
